In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random
import os

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [2]:
search = '암호화폐'

In [3]:
# def get_naver_articles(urls):
#     articles, titles, dates, press = [], [], [], []
    
#     for url in urls:
#         html = requests.get(url, headers=headers)
#         soup = BeautifulSoup(html.text, "html.parser")

#         article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
#         article_titles = [title.text for title in soup.select(".news_tit")]
#         article_infos = [info.text for info in soup.select(".info")]

#         new_urls = []
#         idx = 0

#         while idx < len(article_links) - 1:
#             if 'news.naver.com' in article_links[idx]:
#                 # Check if the next link also contains 'news.naver.com'
#                 if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
#                     new_urls.append(article_links[idx + 1])
#                     idx += 2
#                 else:
#                     new_urls.append(article_links[idx])
#                     idx += 1
#             elif 'news.naver.com' in article_links[idx + 1]:
#                 new_urls.append(article_links[idx + 1])
#                 idx += 2
#             else:
#                 new_urls.append(article_links[idx])
#                 idx += 1

#         # 마지막 요소 처리
#         if idx == len(article_links) - 1:
#             new_urls.append(article_links[idx])

#         date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

#         current_dates = []
#         current_press = []

#         for idx, info in enumerate(article_infos):
#             match = date_pattern.search(info)
#             if match:
#                 current_dates.append(match.group(0))
                
#                 if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
#                     current_press.append(article_infos[idx - 1])

#         articles.extend(new_urls)
#         titles.extend(article_titles)
#         dates.extend(current_dates)
#         press.extend(current_press)

#         time.sleep(random.uniform(1, 1.5))
        
#     return articles, titles, dates, press

In [4]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles_temp = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        # 각 페이지에서 URL이 10개가 아닌 경우 해당 페이지의 URL 출력
        if len(new_urls) % 10 != 0:
            print(f"Warning: {url} contains {len(new_urls)} links instead of 10.")
        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []
        current_titles = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press_name = article_infos[idx - 1]
                    
                    if "CHIEF EXECUTIVE" not in current_press_name:
                        current_press.append(current_press_name)
                        current_titles.append(article_titles_temp[len(current_dates)])
                        current_dates.append(match.group(0))

        articles.extend(new_urls[:len(current_titles)])  # articles length should match with titles
        titles.extend(current_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(random.uniform(1, 1.5))
        
    return articles, titles, dates, press

In [5]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time

In [6]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [55]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=105):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [56]:
# search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")

100%|██████████| 1048/1048 [01:26<00:00, 12.18it/s]

Length of article_urls: 1048
Length of article_titles: 1048
Length of article_dates: 1048
Length of article_press: 1048
Length of contents: 1048
Length of article_time: 1048
            date time                                           title content  \
0     2022.12.22         뉴욕증시 비트코인 푸틴 핵무기 중대발표 쇼크 달러환율 국채금리 국제유가 산...           
1     2022.12.22                            영국, 투자 관리자 면제에 암호화폐 투자 포함           
2     2022.12.22                       프랑스, 올해 암호화폐 웹사이트 2곳 블랙리스트에 추가           
3     2022.12.22                         ['22 결산] 가상자산 혹한기…테라부터 위믹스까지           
4     2022.12.22                         씨티 "이제는 블록(스퀘어) 사라…목표가 90달러"           
...          ...  ...                                             ...     ...   
1043  2022.12.31               암호화폐 시장 2022년 '회색 코뿔소'…테라 붕괴 vs FTX 몰락           
1044  2022.12.31        비트코인과 이더리움 차분한 마무리 예상..새해에 금리와 실적 보수적 접근 필...           
1045  2022.12.31                           비트코인 시세 전망, 새해 앞두고 '소폭 상승'           
1046  2022.12.31

In [53]:
df

,date,time,title,content,press,link
0,2022.10.06,,"EU, 암호화폐 법 MiCA 동의…2024년 적용",,글로벌이코노믹,http://www.g-enews.com/
1,2022.10.06,,"뉴욕증시 비트코인 OPEC ""감산 쇼크"" + 푸틴 핵무기 ""버튼"" 러시아 우크라 확...",,글로벌이코노믹,http://www.g-enews.com/
2,2022.10.06,,"위메이드, 스위스 CLST에 투자 단행…""위믹스3.0 유럽으로 확장""",,AP신문,http://www.apnews.kr
3,2022.10.06,2022-10-06 00:24:22,"돌아가던 레이건함, 북 도발에 동해 유턴",\n\t\t\t 미국의 핵추진 항공모함인 로널드 레이건함(CVN 76)이 5일 다...,A1면 1단,https://n.news.naver.com/mnews/article/025/000...
4,2022.10.06,2022-10-06 01:02:01,"장현국 “스테이블코인, 블록체인 경제 활성화에 필수”",\n‘토큰2049’서 키노트 강연20일 위믹스3.0 메인넷 론칭위믹스 달러도 발행 ...,스포츠월드,https://n.news.naver.com/mnews/article/396/000...
...,...,...,...,...,...,...
3857,2022.11.03,,"트위터에 '바이낸스 페이' 연동 가능성은?...""비트코인 라이트닝 네트워크 도...",,CWN,https://www.cwn.kr/
3858,2022.11.03,None,모자이크배 세계바둑대회 내년 개최 예정,,한게임 바둑,https://sports.news.naver.com/news.nhn?oid=370...
3859,2022.11.03,,"뉴욕증시 비트코인 또 와르르 FOMC ""파월 쇼크"" 국채금리 달러환율 국제유가 ...",,글로벌이코노믹,http://www.g-enews.com/
3860,2022.11.03,,미국 추가 조정 받고 있어..애플과 테슬라 하락..비트코인과 이더리움은 안정...,,매일안전신문,http://peoplesafe.kr


In [54]:
# 데이터 프레임 저장
path = r"C:\\Users\\boyu571\\boyu571_Github\\01_Kakaobank_SKKU_Research_23\\data_gatherting"
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
full_path = os.path.join(path, filename)
df.to_csv(full_path, encoding='utf-8-sig', index=False)


In [38]:

# df.to_csv(r'C:\\Users\\boyu571\\boyu571_Github\\01_Kakaobank_SKKU_Research_23\\data_gatherting\\암호화폐_20200423_20200514.csv', encoding='utf-8-sig', index=False)

In [22]:
article_urls[390:400]

['https://n.news.naver.com/mnews/article/015/0004426402?sid=102',
 'http://www.thepublic.kr/',
 'http://www.dailysecu.com',
 'http://coinreaders.com',
 'https://n.news.naver.com/mnews/article/030/0002906028?sid=105',
 'http://www.epnc.co.kr',
 'http://www.thepublic.kr/',
 'http://www.ciokorea.com',
 'http://www.breaknews.com',
 'http://www.g-enews.com/']

In [ ]:
article_urls.insert(394, "https://www.chiefexe.com/news/ArticleView.asp?listId=Mjc0NHx8bGltaXRfZmFsc2Ug")

In [68]:
df3 = df

In [73]:
df4

,date,time,title,content,press,link
0,2021.03.26,,"[뉴욕증시] 투자흐름 지각변동…""금융 뜨고, 기술 지고""",,아주경제,http://www.ajunews.com
1,2021.03.26,,"뉴욕증시 비트코인 흔들, 바이든 긴급 기자회견 + 국채금리 발작 … 테슬라 애...",,글로벌이코노믹,http://www.g-enews.com/
2,2021.03.26,,"체납자 비트코인도 압류한다…부산시, 은닉자산 찾기 전쟁",,국제신문,http://www.kookje.co.kr/
3,2021.03.26,,"[단독] 특금법 시행 첫날, 개인정보 보호법 위반 거래소 속출",,내외경제tv,http://www.nbntv.co.kr
4,2021.03.26,2021-03-26 03:07:14,CBDC 등장때 비트코인 운명은...“디지털 휴지조각”vs“더 널리 쓰일 것”,"\n제로섬이냐, 공존이냐\n\n\n\n일러스트=김성규\t\t\t\t\t\t\t\t\...",C5면 1단,https://n.news.naver.com/mnews/article/023/000...
...,...,...,...,...,...,...
926,2021.03.31,,인도가 '비트코인 금지법'을 도입하면?,,코인데스크코리아,https://www.coindeskkorea.com/
927,2021.03.31,,[업비트 특징주]비트토렌트 코인 주가 10% 이상 상승,,국제뉴스,http://www.gukjenews.com/
928,2021.03.31,,트리핀의 딜레마,,단비뉴스,http://www.danbinews.com
929,2021.03.31,,NFT,,단비뉴스,http://www.danbinews.com


In [18]:
len(df1)
len(df2)
len(df3)
print(len(df1)+len(df2)+len(df3))

NameError: name 'df2' is not defined

In [74]:
result

,date,time,title,content,press,link
0,2019.03.01,2019-03-01 00:01:01,"[가상화폐 뉴스] 03월 01일 00시 00분 비트코인(0.14%), 이오스(2.0...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
1,2019.03.01,,"암호화페(가상화폐)대표 종목 비트코인시황 상승세,3월 1일 00시 기준 '비트코...",,내외경제tv,http://www.nbntv.co.kr
2,2019.03.01,,비트코인 -0.02%↓ 이더리움 0.40%↑ 리플 0.58%↑ 퀀텀 -1.30%↓ ...,,로이슈,http://www.lawissue.co.kr
3,2019.03.01,,비트코인 0.07%↑ 이더리움 0.40%↑ 리플 0% 라이트코인 0.40%↑ 대시 ...,,로이슈,http://www.lawissue.co.kr
4,2019.03.01,,비트코인 0.09%↑ 이더리움 0.40%↑ 리플 0.58%↑ 대시 -0.04%↓ 에...,,로이슈,http://www.lawissue.co.kr
...,...,...,...,...,...,...
13034,2019.03.31,,[암호화폐뉴스] 31일 가상화폐 대표 종목 상승… 비트코인 23시 17분 기준 시...,,CCTV뉴스,http://www.cctvnews.co.kr
13035,2019.03.31,2019-03-31 23:31:00,"[가상화폐 뉴스] 03월 31일 23시 30분 비트코인(1.15%), 질리카(7.4...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
13036,2019.03.31,,[암호화폐 시황] 비트코인 0.06%↑ 비트코인캐시 0.53%↑ 비트코인골드 0% ...,,빅데이터뉴스,http://www.thebigdata.co.kr/
13037,2019.03.31,,"[세계 거래소별 가상화폐 시황] 31일, 23시 30분 현재",,로이슈,http://www.lawissue.co.kr


In [74]:
result = pd.concat([df1, df2, df3, df4], ignore_index=True)
result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
result

,date,time,title,content,press,link
0,2021.02.01,2021-02-01 00:00:19,'극한의 롤러코스터'…게임스탑發 증시 폭락 우려 확산,\n'극한의 롤러코스터' 게임스탑 격변의 한주한주간 400% 폭등…AMC 등도 주가...,이데일리언론사 선정,https://n.news.naver.com/mnews/article/018/000...
1,2021.02.01,,"뉴욕증시 비트코인 거품 붕괴 조짐, 게임스톱 공매도 일파만파… 테슬라 애플...",,글로벌이코노믹,http://www.g-enews.com/
2,2021.02.01,,"'애로부부' 현영, 결혼생활 10년에도 남편과 뜨거운 이유? ""50살에 다른 매력...",,뉴스웍스,http://www.newsworks.co.kr/
3,2021.02.01,2021-02-01 02:01:33,마디지수 깨진 글로벌 증시…유동성 랠리 끝나나,"\n\t\t\t[이데일리 권소현 기자, 뉴욕=김정남 특파원] 사상 최고치 경신을 거...",A1면 TOP,https://n.news.naver.com/mnews/article/018/000...
4,2021.02.01,2021-02-01 03:03:03,[밀레니얼 톡] 당당하게 ‘돈 이야기’ 하며 삽시다,\n\t\t\t\t\t\t\t\t\t\t90년대생 세 명을 인터뷰했다. 주제는 ‘주...,A29면 1단,https://n.news.naver.com/mnews/article/023/000...
...,...,...,...,...,...,...
12658,2021.03.31,,인도가 '비트코인 금지법'을 도입하면?,,코인데스크코리아,https://www.coindeskkorea.com/
12659,2021.03.31,,[업비트 특징주]비트토렌트 코인 주가 10% 이상 상승,,국제뉴스,http://www.gukjenews.com/
12660,2021.03.31,,트리핀의 딜레마,,단비뉴스,http://www.danbinews.com
12661,2021.03.31,,NFT,,단비뉴스,http://www.danbinews.com


In [75]:
result.to_csv('비트코인_20210201_20210331.csv', encoding='utf-8-sig', index=False)